In [1]:
import warnings
warnings.filterwarnings('ignore')
from category_encoders import LeaveOneOutEncoder,WOEEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(palette = "Dark2")
my_colors = [(0.10588235294117647, 0.6196078431372549, 0.4666666666666667),
  (0.8509803921568627, 0.37254901960784315, 0.00784313725490196)]
pd.set_option('display.max_columns', None)
from itertools import chain, combinations
import math

In [161]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import tensorflow as tensorflow
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv("df_97_columns.csv")
df

,order_item_id,order_date,delivery_date,item_id,size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,order_id,user_age,user_reg_age,order_weekday,delivery_weekday,order_month,delivery_month,order_day,delivery_day,order_week,delivery_week,order_item_count,order_sum,average_item_price_order,order_number_same_item_id,order_number_different_item_id,order_number_same_size,order_number_different_size,order_number_same_item_color,order_number_different_item_color,order_number_same_brand_id,order_number_different_brand_id,order_number_same_item_id_size,order_number_different_item_id_size,order_number_same_item_id_item_color,order_number_different_item_id_item_color,order_number_same_size_item_color,order_number_different_size_item_color,order_number_same_size_brand_id,order_number_different_size_brand_id,order_number_same_item_color_brand_id,order_number_different_item_color_brand_id,order_number_same_item_id_size_item_color,order_number_different_item_id_size_item_color,order_number_same_size_item_color_brand_id,order_number_different_size_item_color_brand_id,order_item_id_nunique,order_size_nunique,order_brand_id_nunique,order_item_color_nunique,order_item_id_color_nunique,order_item_id_size_nunique,order_brand_id_color_nunique,order_brand_id_size_nunique,order_brand_id_item_id_nunique,item_price_item_id_mean,item_price_item_id_std,item_price_item_id_min,item_price_item_id_max,item_price_item_id_sum,item_price_item_id_count,item_price_item_id_median,item_price_item_id_mad,item_price_user_id_mean,item_price_user_id_std,item_price_user_id_min,item_price_user_id_max,item_price_user_id_sum,item_price_user_id_count,item_price_user_id_median,item_price_user_id_mad,item_price_brand_id_mean,item_price_brand_id_std,item_price_brand_id_min,item_price_brand_id_max,item_price_brand_id_sum,item_price_brand_id_count,item_price_brand_id_median,item_price_brand_id_mad,mode_item_id,mode_size,mode_brand_id,mode_item_color,price-item_price_item_id_mean,price-item_price_item_id_min,price-item_price_item_id_max,price-item_price_user_id_mean,price-item_price_user_id_min,price-item_price_user_id_max,price-item_price_brand_id_mean,price-item_price_brand_id_min,price-item_price_brand_id_max
0,1,2012-04-01,2012-04-03,186,s,denim,25,69.90,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,0.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,2,2,2,3,1,1,1,1,1,69.47,3.41,39.90,69.90,21606.68,311,69.90,0.83,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,55.23,13.65,24.9,79.90,282536.19,5116,59.90,11.66,71,s,20,white,-0.43,-30.0,0.0,-7.25,-69.90,20.00,-14.67,-45.00,10.0
1,2,2012-04-01,2012-04-03,71,unsized,ocher,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0
2,3,2012-04-01,2012-04-03,71,unsized,curry,21,69.95,794,Mrs,1965-01-06,Bad-Wue,2011-04-25,1.0,2,2012-04-01_794,47,342,6,1,4,4,1,3,13,14,3,209.8,69.93,2,1,2,1,1,2,2,1,2,1,1,2,1,2,2,1,1,2,1,2,1,2,2,2,2,3,2,1,2,1,1,57.95,13.28,34.95,69.95,59108.70,1020,69.95,12.35,62.65,24.51,0.0,89.9,689.1,11,69.9,16.52,74.54,30.32,24.9,179.95,478991.63,6426,69.95,22.66,71,s,20,white,-12.00,-35.0,0.0,-7.30,-69.95,19.95,4.59,-45.05,110.0
3,4,2012-04-02,2012-04-06,22,s,green,14,39.90,808,Mrs,1959-11-09,Saxony,2012-01-04,0.0,4,2012-04-02_808,52,89,0,4,4,4,2,6,14,14,1,39.9,39.90,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,1,1,1,1,1,1,1,31.10,8.68,19.90,39.90,68669.20,2208,29.90,8.41,39.90,0.00,39.9,39.9,39.9,1,39.9,0.00,55.16,19.34,0.0,89.90,629751.00,11417,49.90,16.21,22,s,14,green,-8.80,-20.0,0.0,0.00,0.00,0.00,15.26,-39.90,50.0
4,5,2012-04-02,2012-04-06,151,s,black,53,29.90,825,Mrs,1964-07-11,S-Holstein,2011-02-

In [247]:
# Create a class for all the models
class Model(object):
    def __init__(self, df:dict):
        self.df = df
        
        
    def split_data(self, historic:bool)->tuple:
        """
        Split the data into train and test sets, depending on whether the encdoder needs to be fited on the full data or not.

        Parameters
        ----------
        historic : bool
            False if the encoder needs to be fited on the Months April to February and to transform March
            True if the encoder needs to be fited on the Months April to December and to transform January to March

        Returns
        -------
        tuple
            df_train : dict
                Dataframe with training data.
            df_test : dict
                Dataframe with testing data.
        """
        # Get list of the months to train, remove months on which to test        
        if historic:
            months_to_train = list(range(1,13))
            # remove months 12,1,2,3 from the train set
            months_to_train = [i for i in months_to_train if i not in [12,1,2,3]]
        else:
            months_to_train = list(range(1,13))
            months_to_train.remove(3)
        k = max(self.df["order_item_id"])-1
        # Split into train and test. "~" in front of a variable means "not"
        df_train = self.df.loc[:k][self.df.loc[:k,"order_month"].isin(months_to_train)]
        df_test = self.df.loc[:k][~self.df.loc[:k,"order_month"].isin(months_to_train)]
        # Get the validation set
        # df_valid = self.df.iloc[k+1:, :]
        # Drop unnecessary columns
        columns_to_drop = ["order_date", "delivery_date", "user_dob", "user_reg_date", "order_id","order_item_id"]
        df_train.drop(columns_to_drop, axis=1, inplace=True)
        df_test.drop(columns_to_drop, axis=1, inplace=True)
        return df_train, df_test
    
    def LOE_Encoder(self, df_train:dict, df_test:dict, columns:list ,sig:float)->tuple:
        """
        Leave One Out Encoder to calculate the response variable for each category.

        Parameters
        ----------
        df_train : dict
            Dataframe with training data.
        df_test : dict
            Dataframe with testing data.
        columns : list
            Categorical columns to encode.
        sig : float
            Random noise added to the response variable.

        Returns
        -------
        tuple
            _description_
        """              
        encoder = LeaveOneOutEncoder(cols=columns, return_df=True,sigma=sig)
        df_encode_train = encoder.fit_transform(df_train.drop(["return"],axis=1),df_train[["return"]]).round(3)
        df_encode_test = encoder.transform(df_test.drop(["return"],axis=1)).round(3)
        df_encode_train , df_encode_test = df_encode_train.join(df_train[["return"]]), df_encode_test.join(df_test[["return"]])
        return df_encode_train, df_encode_test, encoder
    
    def neural_network(self, df_train:dict, df_test:dict, n_layers:int, n_nodes:int, dropout:list, activation:str, optimizer:str, loss:str, metrics:list, epochs:int, batch_size:int,verbose:int)->object:
        """
        Neural network model to predict whether an item will be returned or not.

        Parameters
        ----------
        df_train : dict
            Dataframe with training data.
        df_test : dict
            Dataframe with testing data.
        n_layers : int
            Number of layers in the neural network
        n_nodes : int
            Number of nodes in each layer
        dropout : list
            List of dropout rates for each layer
        activation : str
            Activation function for each layer except the last one
        optimizer : str
            Optimizer for the neural network
        loss : str
            Loss function for the neural network
        metrics : str
            List of metrics for the neural network
        epochs : int
            Number of epochs for the neural network
        batch_size : int
            Size of the batch
        verbose : int
            Whether to print the progress of the neural network

        Returns
        -------
        model: object
            The Neural network model
        Y_pred: array
            Array of floats with the predictions of the neural network
        mae: float
            Mean absolute error on the testing set
        """         
        X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
        X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
        X_train = StandardScaler().fit_transform(X_train)
        X_test = StandardScaler().fit_transform(X_test)
        
        model = Sequential()
        for i in range(n_layers):
            if i == 0:
                model.add(Dense(n_nodes[i], input_dim=X_train.shape[1], activation=activation))
            else:
                model.add(Dense(n_nodes[i], activation=activation))
            model.add(Dropout(dropout[i]))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,verbose=verbose,validation_data=(X_test, Y_test))
        Y_pred = model.predict(X_test).round(4)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae,
    
    def xgboost(self, df_train:dict, df_test:dict, params:dict, verbose:int):
        X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
        X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
        model = XGBClassifier(**params)
        model.fit(X_train, Y_train, eval_metric='mae', eval_set=[(X_test, Y_test)],early_stopping_rounds = 20,verbose = verbose)
        Y_pred = model.predict(X_test)
        Y_pred_prob = model.predict_proba(X_test)[:,1].round(4)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae, Y_pred_prob
    
    def catboost(self, df_train:dict, df_test:dict, params:dict, verbose:bool):
        X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
        X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
        model = CatBoostClassifier(**params)
        model.fit(X_train, Y_train, eval_set=(X_test, Y_test), use_best_model=True, verbose=verbose,early_stopping_rounds = 20)
        Y_pred = model.predict(X_test)
        Y_pred_prob = model.predict_proba(X_test)[:,1].round(4)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae, Y_pred_prob
    
    def lightgmb(self, df_train:dict, df_test:dict, params:dict, verbose:int):
        X_train, Y_train = df_train.drop(["return"],axis=1), df_train["return"]
        X_test, Y_test = df_test.drop(["return"],axis=1), df_test["return"]
        model = LGBMClassifier(**params)
        model.fit(X_train, Y_train, verbose=verbose)
        Y_pred = model.predict(X_test)
        mae = mean_absolute_error(Y_test, Y_pred)
        return model, Y_pred, mae, Y_pred_prob
    

In [248]:
Model = Model(df)

In [3]:
low_cardinality_cat_cols = ["size","item_color","user_title","user_state"]
high_cardinality_cat_cols = ["item_id","brand_id","user_id"]
cat_cols = low_cardinality_cat_cols + high_cardinality_cat_cols
cat_cols += ["mode_item_id","mode_size","mode_brand_id","mode_item_color",]

In [219]:
df_train,df_test = Model.split_data(False)

In [220]:
df_train_encode,df_test_encode, encoder = Model.LOE_Encoder(df_train,df_test,cat_cols,0.1)

In [236]:
neural_network_params = {"n_layers":2, "n_nodes":[64,64], "dropout":[0.1,0.1], "activation":"relu", "optimizer":"adam", "loss":"mae", "metrics":["mae"], "epochs":5, "batch_size":256, "verbose":1}
neural_network_model, nn_Y_pred, nn_mae = Model.neural_network(df_train_encode, df_test_encode, **neural_network_params)

Epoch 1/5
1608/1608 [==============================] - 4s 2ms/step - loss: 0.3298 - mae: 0.3298 - val_loss: 0.3655 - val_mae: 0.3655
Epoch 2/5
1608/1608 [==============================] - 4s 2ms/step - loss: 0.3178 - mae: 0.3178 - val_loss: 0.3656 - val_mae: 0.3656
Epoch 3/5
1608/1608 [==============================] - 3s 2ms/step - loss: 0.3165 - mae: 0.3165 - val_loss: 0.3634 - val_mae: 0.3634
Epoch 4/5
1608/1608 [==============================] - 3s 2ms/step - loss: 0.3158 - mae: 0.3158 - val_loss: 0.3641 - val_mae: 0.3641
Epoch 5/5
1608/1608 [==============================] - 4s 2ms/step - loss: 0.3148 - mae: 0.3148 - val_loss: 0.3637 - val_mae: 0.3637


In [237]:
print("Neural Network MAE:",nn_mae)

Neural Network MAE: 0.36371597794316374


In [238]:
print(nn_Y_pred_prob)

[[0.9999986 ]
 [0.7806934 ]
 [0.08946997]
 ...
 [0.13059604]
 [0.99991786]
 [0.9999994 ]]


In [249]:
xgb_params = { "max_depth":9, "learning_rate":0.16, "n_estimators":50, "objective":"binary:logistic","random_state":42}
xgb_model, xgb_Y_pred, xgb_mae, xgb_Y_pred_prob = Model.xgboost(df_train_encode, df_test_encode, xgb_params, 1)
print(xgb_mae)

[0]	validation_0-mae:0.48867
[1]	validation_0-mae:0.47886
[2]	validation_0-mae:0.47101
[3]	validation_0-mae:0.46392
[4]	validation_0-mae:0.45799
[5]	validation_0-mae:0.45239
[6]	validation_0-mae:0.44761
[7]	validation_0-mae:0.44384
[8]	validation_0-mae:0.44031
[9]	validation_0-mae:0.43723
[10]	validation_0-mae:0.43454
[11]	validation_0-mae:0.43250
[12]	validation_0-mae:0.43079
[13]	validation_0-mae:0.42916
[14]	validation_0-mae:0.42756
[15]	validation_0-mae:0.42642
[16]	validation_0-mae:0.42532
[17]	validation_0-mae:0.42431
[18]	validation_0-mae:0.42328
[19]	validation_0-mae:0.42241
[20]	validation_0-mae:0.42183
[21]	validation_0-mae:0.42070
[22]	validation_0-mae:0.42018
[23]	validation_0-mae:0.41973
[24]	validation_0-mae:0.41934
[25]	validation_0-mae:0.41819
[26]	validation_0-mae:0.41773
[27]	validation_0-mae:0.41731
[28]	validation_0-mae:0.41700
[29]	validation_0-mae:0.41676
[30]	validation_0-mae:0.41642
[31]	validation_0-mae:0.41633
[32]	validation_0-mae:0.41624
[33]	validation_0-ma

In [250]:
xgb_Y_pred_prob

array([0.5312, 0.4375, 0.511 , ..., 0.3551, 0.3768, 0.4492], dtype=float32)

In [244]:
catboost_params = { "depth":9, "random_seed":42, "iterations":100, "loss_function":"Logloss", "verbose":1,"learning_rate":0.15}
catboost_model, catboost_Y_pred, catboost_mae, catboost_Y_pred_prob = Model.catboost(df_train_encode, df_test_encode, catboost_params, 1)
print(catboost_mae)

0:	learn: 0.6606851	test: 0.6718289	best: 0.6718289 (0)	total: 222ms	remaining: 22s
1:	learn: 0.6384882	test: 0.6583944	best: 0.6583944 (1)	total: 518ms	remaining: 25.4s
2:	learn: 0.6224369	test: 0.6488076	best: 0.6488076 (2)	total: 718ms	remaining: 23.2s
3:	learn: 0.6090702	test: 0.6381527	best: 0.6381527 (3)	total: 940ms	remaining: 22.5s
4:	learn: 0.6005937	test: 0.6331238	best: 0.6331238 (4)	total: 1.11s	remaining: 21.1s
5:	learn: 0.5931683	test: 0.6278579	best: 0.6278579 (5)	total: 1.39s	remaining: 21.8s
6:	learn: 0.5879438	test: 0.6252581	best: 0.6252581 (6)	total: 1.67s	remaining: 22.2s
7:	learn: 0.5835347	test: 0.6209040	best: 0.6209040 (7)	total: 1.88s	remaining: 21.6s
8:	learn: 0.5804200	test: 0.6190774	best: 0.6190774 (8)	total: 2.21s	remaining: 22.3s
9:	learn: 0.5780524	test: 0.6180304	best: 0.6180304 (9)	total: 2.51s	remaining: 22.6s
10:	learn: 0.5763433	test: 0.6172690	best: 0.6172690 (10)	total: 2.74s	remaining: 22.2s
11:	learn: 0.5745531	test: 0.6167325	best: 0.6167325 (

In [109]:
# depth 10, lr 0,16
print(catboost_mae)

0.33114608161272724


In [252]:
catboost_Y_pred

array([1., 0., 0., ..., 0., 1., 1.])

In [246]:
catboost_Y_pred_prob

array([0.55246634, 0.40882318, 0.41100704, ..., 0.42606847, 0.51618666,
       0.56636642])

In [253]:
mean_absolute_error(df_test_encode["return"], catboost_Y_pred)

0.3313470981822359

In [260]:
mean_list = []
error_best = 1
for i in range(1,101):
    first_term = catboost_Y_pred_prob*(i/100)
    second_term = xgb_Y_pred_prob*((100-i)/100)
    sum = first_term + second_term
    error = mean_absolute_error(df_test_encode["return"], sum.round())
    if error < error_best:
        error_best = error
        best_i = i
    mean_list.append(error)

In [267]:
error_best

0.33105993165436637